# Clone Github Repo

In [1]:
!git clone https://github.com/sirius-ai/LPRNet_Pytorch.git

fatal: destination path 'LPRNet_Pytorch' already exists and is not an empty directory.


In [2]:
!wget https://github.com/sirius-ai/LPRNet_Pytorch/raw/master/weights/Final_LPRNet_model.pth -O weights/Final_LPRNet_model.pth

weights/Final_LPRNet_model.pth: No such file or directory


#Installation

In [3]:
!python3 -m  pip install mlc-ai-cpu -f https://mlc.ai/wheels

Looking in links: https://mlc.ai/wheels


In [4]:
!pip install --upgrade tvm

In [5]:
print('Installing torchprofile...')
!pip install torchprofile 1>/dev/null
print('All required packages have been successfully installed!')

Installing torchprofile...
All required packages have been successfully installed!


In [6]:
!pip install torch torchvision numpy opencv-python-headless

In [7]:
!pip install onnx

In [8]:
%cd /content/LPRNet_Pytorch

/content/LPRNet_Pytorch


#Import necessary modules

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import sys
import numpy as np
from torch.utils.data import DataLoader
from model.LPRNet import LPRNet
from data.load_data import LPRDataLoader, CHARS


from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
from PIL import Image, ImageDraw, ImageFont
from model.LPRNet import build_lprnet, small_basic_block
# import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import cv2
import os

import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.sparse as sparse
import copy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import argparse
import os
from torch.autograd import Variable
from torch.utils.data import *
from data.load_data import LPRDataLoader, CHARS
from model.LPRNet import build_lprnet

import torch.quantization
import torch.onnx

#LPRNet

In [10]:
lpr_max_len = 8
# Set up the device
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
# Initialize the model
lprnet = LPRNet(lpr_max_len=lpr_max_len, class_num=len(CHARS), dropout_rate=0, phase="eval")

# Load the pre-trained weights
weights_path = 'weights/Final_LPRNet_model.pth'
state_dict = torch.load(weights_path, map_location=device)
lprnet.load_state_dict(state_dict)

# Move the model to the appropriate device
lprnet = lprnet.to(device)

# Set the model to evaluation mode
lprnet.eval()

print("LPRNet model loaded successfully.")
print(f"Model is on device: {next(lprnet.parameters()).device}")

LPRNet model loaded successfully.
Model is on device: cpu


<ipython-input-10-eaddd8717b46>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path, map_location=device)


In [11]:
test_img_dirs = ["data/test"]
test_dataset = LPRDataLoader(test_img_dirs, imgSize=(94, 24), lpr_max_len=lpr_max_len)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

In [12]:
def collate_fn(batch):
    imgs = []
    labels = []
    lengths = []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

In [13]:
import argparse

def get_parser_test():
    parser = argparse.ArgumentParser(description='parameters to train net')
    parser.add_argument('--img_size', default=[94, 24], help='the image size')
    parser.add_argument('--test_img_dirs', default="./data/test", help='the test images path')
    parser.add_argument('--dropout_rate', default=0, help='dropout rate.')
    parser.add_argument('--lpr_max_len', default=8, help='license plate number max length.')
    parser.add_argument('--test_batch_size', default=100, help='testing batch size.')
    parser.add_argument('--phase_train', default=False, type=bool, help='train or test phase flag.')
    parser.add_argument('--num_workers', default=8, type=int, help='Number of workers used in dataloading')
    parser.add_argument('--cuda', default=False, type=bool, help='Use cuda to train model')
    parser.add_argument('--show', default=False, type=bool, help='show test image and its predict result or not.')
    parser.add_argument('--pretrained_model', default='./weights/Final_LPRNet_model.pth', help='pretrained base model')

    # Parse arguments with an empty list to avoid reading from sys.argv
    args = parser.parse_args(args=[])

    return args

In [14]:
def Greedy_Decode_Eval(Net, datasets, args):
    # TestNet = Net.eval()
    epoch_size = len(datasets) // args.test_batch_size
    batch_iterator = iter(DataLoader(datasets, args.test_batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn))

    Tp = 0
    Tn_1 = 0
    Tn_2 = 0
    t1 = time.time()
    for i in range(epoch_size):
        # load train data
        images, labels, lengths = next(batch_iterator)
        start = 0
        targets = []

        for length in lengths:
            label = labels[start:start+length]
            targets.append(label)
            start += length
        targets = np.array([el.numpy() for el in targets])
        imgs = images.numpy().copy()

        if args.cuda:
            images = Variable(images.cuda())
        else:
            images = Variable(images)

        # forward
        # forward
        with torch.no_grad():
          prebs = Net(images)
        # greedy decode
        prebs = prebs.cpu().detach().numpy()
        preb_labels = list()
        for i in range(prebs.shape[0]):
            preb = prebs[i, :, :]
            preb_label = list()
            for j in range(preb.shape[1]):
                preb_label.append(np.argmax(preb[:, j], axis=0))
            no_repeat_blank_label = list()
            pre_c = preb_label[0]
            if pre_c != len(CHARS) - 1:
                no_repeat_blank_label.append(pre_c)
            for c in preb_label: # dropout repeate label and blank label
                if (pre_c == c) or (c == len(CHARS) - 1):
                    if c == len(CHARS) - 1:
                        pre_c = c
                    continue
                no_repeat_blank_label.append(c)
                pre_c = c
            preb_labels.append(no_repeat_blank_label)
        for i, label in enumerate(preb_labels):
            # show image and its predict label
            if args.show:
                show(imgs[i], label, targets[i])
            if len(label) != len(targets[i]):
                Tn_1 += 1
                continue
            if (np.asarray(targets[i]) == np.asarray(label)).all():
                Tp += 1
            else:
                Tn_2 += 1
    Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)

    t2 = time.time()
    print("[Info] Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))
    return Acc, (t2 - t1) / len(datasets)

In [15]:
args = get_parser_test()
baseline_accuracy, baseline_test_speed = Greedy_Decode_Eval(lprnet, test_dataset, args)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[Info] Test Speed: 0.20374352216720581s 1/1000]


#Pruning

In [16]:
def structured_prune_conv(module, name, amount):
    if not isinstance(module, nn.Conv2d):
        return

    weight = getattr(module, name)
    output_channels = weight.shape[0]
    num_prune = int(output_channels * amount)

    # Calculate L1-norm for each filter
    l1_norm = torch.sum(torch.abs(weight.view(output_channels, -1)), dim=1)

    # Get indices of filters to keep (those with highest L1-norm)
    top_k = torch.topk(l1_norm, output_channels - num_prune, largest=True, sorted=False)
    mask = torch.zeros_like(l1_norm)
    mask[top_k.indices] = 1

    # Create a sparse mask
    sparse_mask = mask.to_sparse()

    # Apply the mask to the weight tensor
    pruned_weight = weight * mask.view(-1, 1, 1, 1)

    # Set the pruned weights back to the module
    setattr(module, name, nn.Parameter(pruned_weight))

    # Save the mask for future reference
    module.register_buffer(f'{name}_mask', sparse_mask)

In [17]:
lprnet_copy = copy.deepcopy(lprnet)

print("LPRNet model deepcopied successfully.")
print(f"Model is on device: {next(lprnet_copy.parameters()).device}")

LPRNet model deepcopied successfully.
Model is on device: cpu


In [18]:
def prune_model(model, amount=0.1):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            structured_prune_conv(module, 'weight', amount)
    return model

In [19]:
# Prune the model
pruned_lprnet = prune_model(lprnet_copy, amount=0.01)

print("Finished pruning the model")

Finished pruning the model


In [20]:
args = get_parser_test()
pruned_accuracy, pruned_test_speed = Greedy_Decode_Eval(pruned_lprnet, test_dataset, args)

[Info] Test Speed: 0.1872528645992279s 1/1000]


In [21]:
pruned_lprnet

LPRNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (4): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): ReLU()
        (4): Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (5): ReLU()
        (6): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2), padding=0, dilation=1, ceil_mode=False)
    (8): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 64, 

## Fine-tuning attempt

In [22]:
# import argparse

# def get_parser_train():
#     parser = argparse.ArgumentParser(description='parameters to train net')
#     parser.add_argument('--max_epoch', default=4, help='epoch to train the network')
#     parser.add_argument('--img_size', default=[94, 24], help='the image size')
#     parser.add_argument('--train_img_dirs', default="./data/train", help='the train images path')
#     parser.add_argument('--test_img_dirs', default="./data/test", help='the test images path')
#     parser.add_argument('--dropout_rate', default=0.5, help='dropout rate.')
#     parser.add_argument('--learning_rate', default=0.001, help='learning rate')
#     parser.add_argument('--lpr_max_len', default=8, help='license plate number max length')
#     parser.add_argument('--train_batch_size', default=128, help='training batch size')
#     parser.add_argument('--test_batch_size', default=128, help='testing batch size')
#     parser.add_argument('--phase_train', default=True, type=bool, help='train or test phase flag')
#     parser.add_argument('--num_workers', default=8, type=int, help='Number of workers used in dataloading')
#     parser.add_argument('--cuda', default=True, type=bool, help='Use cuda to train model')
#     parser.add_argument('--resume_epoch', default=0, type=int, help='resume iter for retraining')
#     parser.add_argument('--save_interval', default=1000, type=int, help='interval for save model state dict')
#     parser.add_argument('--test_interval', default=2000, type=int, help='interval for evaluate')
#     parser.add_argument('--momentum', default=0.9, type=float, help='momentum')
#     parser.add_argument('--weight_decay', default=2e-5, type=float, help='Weight decay for SGD')
#     parser.add_argument('--lr_schedule', default=[4, 8], help='schedule for learning rate.')
#     parser.add_argument('--save_folder', default='./weights/', help='Location to save checkpoint models')
#     parser.add_argument('--pretrained_model', default='./weights/Final_LPRNet_model.pth', help='pretrained base model')
#     args = parser.parse_args(args=[])

#     return args


In [23]:
# def train(lprnet):
#     args = get_parser_train()

#     # Load the pruned model
#     # lprnet = torch.load('path_to_your_pruned_model.pth')
#     # lprnet = lprnet.to(device)

#     train_dataset = LPRDataLoader(args.train_img_dirs.split(','), args.img_size, args.lpr_max_len)
#     trainloader = DataLoader(train_dataset, batch_size=args.train_batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn)

#     criterion = nn.CTCLoss(blank=len(CHARS)-1, reduction='mean')
#     optimizer = optim.RMSprop(lprnet.parameters(), lr=args.learning_rate * 0.1, alpha=0.9, eps=1e-08, weight_decay=args.weight_decay, momentum=args.momentum)
#     train_loss = 0
#     cnt = 0
#     for epoch in range(args.max_epoch):
#         for i, (imgs, labels, lengths) in enumerate(trainloader):
#             imgs, labels = imgs.to(device), labels.to(device)
#             optimizer.zero_grad()
#             transfer = lprnet(imgs)
#             log_probs = transfer.permute(2, 0, 1)
#             log_probs = log_probs.log_softmax(2).requires_grad_()
#             input_lengths = torch.full((log_probs.size(1),), log_probs.size(0), dtype=torch.long)
#             target_lengths = torch.from_numpy(np.array(lengths)).type(torch.long)
#             loss = criterion(log_probs, labels, input_lengths, target_lengths)
#             loss.backward()

#             # Apply masks to gradients
#             for name, module in lprnet.named_modules():
#                 if isinstance(module, nn.Conv2d) and hasattr(module, 'weight_mask'):
#                     module.weight.grad *= module.weight_mask

#             optimizer.step()

#             train_loss += loss.item()
#             cnt += 1

#             if i % 100 == 0:
#                 print(f'Epoch [{epoch+1}/{args.max_epoch}], Iter [{i+1}/{len(trainloader)}], Loss: {train_loss/cnt:.4f}')
#                 train_loss = 0
#                 cnt = 0

#         # Save model
#         if (epoch + 1) % args.save_interval == 0:
#             test_args = get_parser_test()
#             print(Greedy_Decode_Eval(lprnet, test_dataset, test_args))
#             #torch.save(lprnet.state_dict(), args.save_folder + 'LPRNet_' + str(epoch + 1) + '.pth')

#     print(Greedy_Decode_Eval(lprnet, test_dataset, args))
#     # Final save
#     # torch.save(lprnet.state_dict(), args.save_folder + 'Final_LPRNet_model.pth')

In [24]:
# device = "cpu"
# train(lprnet_copy)

#Quantization

In [25]:
class QuantizableLPRNet(nn.Module):
    def __init__(self, original_model):
        super().__init__()
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
        self.model = self.modify_model(original_model)

    def modify_model(self, model):
        for name, module in model.named_children():
            if isinstance(module, nn.MaxPool3d):
                class ContiguousModule(nn.Module):
                    def forward(self, x):
                        return x.contiguous()

                setattr(model, name, nn.Sequential(
                    self.dequant,
                    ContiguousModule(),
                    module,
                    self.quant
                ))
            elif isinstance(module, nn.Sequential):
                self.modify_model(module)
        return model

    def forward(self, x):
        x = self.quant(x)
        keep_features = list()
        for i, layer in enumerate(self.model.backbone.children()):
            # print()
            x = layer(x)

            if i in [2, 6, 13, 22]:
                keep_features.append(x)

        global_context = list()
        for i, f in enumerate(keep_features):
            if i in [0, 1]:
                f = nn.AvgPool2d(kernel_size=5, stride=5)(f)
            if i in [2]:
                f = nn.AvgPool2d(kernel_size=(4, 10), stride=(4, 2))(f)

            # Dequantize before torch.pow
            f = self.dequant(f)
            f_pow = torch.pow(f, 2)
            f_mean = torch.mean(f_pow)
            f = torch.div(f, f_mean)
            # Quantize after torch.div
            f = self.quant(f)

            global_context.append(f)

        x = torch.cat(global_context, 1)
        x = self.model.container(x)
        logits = torch.mean(x, dim=2)
        x = self.dequant(logits).contiguous()
        return x

In [26]:
# prompt: generate a deepcopy of the pruned_lprnet model

pruned_lprnet_copy = copy.deepcopy(lprnet)
print("Pruned LPRNet model deepcopied successfully.")
print(f"Model is on device: {next(pruned_lprnet_copy.parameters()).device}")

Pruned LPRNet model deepcopied successfully.
Model is on device: cpu


In [27]:
def prepare_for_quantization(model):
    model.eval()
    model = QuantizableLPRNet(model)
    # Function to fuse Conv, BN, and ReLU layers in a Sequential module
    def fuse_sequential(sequential):
        modules_to_fuse = []
        last_conv_index = None

        for i in range(len(sequential)):
            if isinstance(sequential[i], small_basic_block):
                # Find the last Conv2d layer in the small_basic_block
                for j, layer in enumerate(sequential[i].block):
                    if isinstance(layer, nn.Conv2d):
                        last_conv_index = f"{i}.block.{j}"
            elif last_conv_index is not None:
                if isinstance(sequential[i], nn.BatchNorm2d) and \
                  isinstance(sequential[i+1], nn.ReLU):
                    modules_to_fuse.append([last_conv_index, str(i), str(i+1)])
                    last_conv_index = None
                else:
                    last_conv_index = None
            elif i < len(sequential) - 2:
                if isinstance(sequential[i], nn.Conv2d) and \
                  isinstance(sequential[i+1], nn.BatchNorm2d) and \
                  isinstance(sequential[i+2], nn.ReLU):
                    modules_to_fuse.append([str(i), str(i+1), str(i+2)])
                elif isinstance(sequential[i], nn.Conv2d) and \
                    isinstance(sequential[i+1], nn.BatchNorm2d):
                    modules_to_fuse.append([str(i), str(i+1)])

        if modules_to_fuse:
            torch.quantization.fuse_modules(sequential, modules_to_fuse, inplace=True)

    # Fuse layers in the backbone
    fuse_sequential(model.model.backbone)

    # Fuse layers in the container
    fuse_sequential(model.model.container)

    return model

In [28]:
def quantize_model(model):
    torch.backends.quantized.engine = 'fbgemm'

    model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
    torch.quantization.prepare(model, inplace=True)


    with torch.no_grad():
        model.eval()
        for i, (imgs, labels, lengths) in enumerate(test_loader):
            if i > 100:  # Calibrate with 100 batches
                break
            imgs = imgs.to('cpu')  # Quantization requires CPU tensors
            a = model(imgs)

    # Convert to quantized model
    torch.quantization.convert(model, inplace=True)

    return model

In [29]:
# Prepare the pruned model for quantization
pruned_lprnet_copy = prepare_for_quantization(pruned_lprnet_copy)

# Move the model to CPU (quantization requires CPU)
pruned_lprnet_copy = pruned_lprnet_copy.cpu()

# Quantize the model
quantized_lprnet = quantize_model(pruned_lprnet_copy)
quantized_lprnet = quantized_lprnet.cpu()

print("Finished quantizing the model")

/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Finished quantizing the model


In [30]:
def Greedy_Decode_Eval_quantized(Net, datasets, args):
    Net.eval()
    Net = Net.cpu()
    epoch_size = len(datasets) // args.test_batch_size
    batch_iterator = iter(DataLoader(datasets, args.test_batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn))

    Tp = 0
    Tn_1 = 0
    Tn_2 = 0
    t1 = time.time()
    for i in range(epoch_size):
        # load train data
        images, labels, lengths = next(batch_iterator)
        images = images.cpu().contiguous()
        start = 0
        targets = []

        for length in lengths:
            label = labels[start:start+length]
            targets.append(label)
            start += length
        targets = np.array([el.numpy() for el in targets])
        imgs = images.numpy().copy()

        images = Variable(images)

        # forward
        # forward
        with torch.no_grad():
          prebs = Net(images)
        # greedy decode
        prebs = prebs.cpu().detach().numpy()
        preb_labels = list()
        for i in range(prebs.shape[0]):
            preb = prebs[i, :, :]
            preb_label = list()
            for j in range(preb.shape[1]):
                preb_label.append(np.argmax(preb[:, j], axis=0))
            no_repeat_blank_label = list()
            pre_c = preb_label[0]
            if pre_c != len(CHARS) - 1:
                no_repeat_blank_label.append(pre_c)
            for c in preb_label: # dropout repeate label and blank label
                if (pre_c == c) or (c == len(CHARS) - 1):
                    if c == len(CHARS) - 1:
                        pre_c = c
                    continue
                no_repeat_blank_label.append(c)
                pre_c = c
            preb_labels.append(no_repeat_blank_label)
        for i, label in enumerate(preb_labels):
            # show image and its predict label
            if args.show:
                show(imgs[i], label, targets[i])
            if len(label) != len(targets[i]):
                Tn_1 += 1
                continue
            if (np.asarray(targets[i]) == np.asarray(label)).all():
                Tp += 1
            else:
                Tn_2 += 1
    Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
    t2 = time.time()
    return Acc, (t2 - t1) / len(datasets)

In [31]:
import traceback

In [32]:
try:
    args = get_parser_test()
    quantized_accuracy, quantized_test_speed = Greedy_Decode_Eval_quantized(quantized_lprnet, test_dataset, args)
except Exception as e:
    print(traceback.format_exc())

In [33]:
quantized_lprnet

QuantizableLPRNet(
  (quant): Quantize(scale=tensor([0.4502]), zero_point=tensor([2]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (model): LPRNet(
    (backbone): Sequential(
      (0): QuantizedConvReLU2d(3, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.1549132764339447, zero_point=0)
      (1): Identity()
      (2): Identity()
      (3): Sequential(
        (0): DeQuantize()
        (1): ContiguousModule()
        (2): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
        (3): Quantize(scale=tensor([0.4502]), zero_point=tensor([2]), dtype=torch.quint8)
      )
      (4): small_basic_block(
        (block): Sequential(
          (0): QuantizedConv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), scale=0.42272183299064636, zero_point=71)
          (1): ReLU()
          (2): QuantizedConv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), scale=0.8551036715507507, zero_point=69, padding=(1, 0))
          (3): ReLU()
          (4): QuantizedConv2d

#Baseline - TVM

In [34]:
import torch
import tvm
from tvm import relay

In [35]:
args = get_parser_test()
test_loader = DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=args.num_workers, collate_fn=collate_fn)
sample_input, _, _ = next(iter(test_loader))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [36]:
input_shape = sample_input.shape

In [37]:
quantized_lprnet = quantized_lprnet.eval()
scripted_model = torch.jit.trace(quantized_lprnet, sample_input).eval()

In [38]:
input_name = "input"
shape_list = [(input_name, input_shape)]
mod, params = relay.frontend.from_pytorch(scripted_model, shape_list)

In [39]:
target = tvm.target.Target("llvm", host="llvm")
dev = tvm.cpu(0)

In [40]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

In [41]:
from tvm.contrib import graph_executor

dtype = "float32"
module = graph_executor.GraphModule(lib["default"](dev))

In [42]:
def Greedy_Decode_Eval_TVM(tvm_module, datasets, args):
    epoch_size = len(datasets) // args.test_batch_size
    batch_iterator = iter(DataLoader(datasets, args.test_batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn))

    Tp = 0
    Tn_1 = 0
    Tn_2 = 0
    t1 = time.time()

    for i in range(epoch_size):
        # Load test data
        images, labels, lengths = next(batch_iterator)
        start = 0
        targets = []

        for length in lengths:
            label = labels[start:start+length]
            targets.append(label)
            start += length
        targets = np.array([el.numpy() for el in targets])
        imgs = images.numpy().copy()

        tvm_input = tvm.nd.array(images.numpy())
        # Set input for TVM module
        tvm_module.set_input("input", images.numpy())

        # Run inference
        tvm_module.run()

        # Get output
        prebs = tvm_module.get_output(0).asnumpy()

        # Greedy decode
        preb_labels = list()
        for i in range(prebs.shape[0]):
            preb = prebs[i, :, :]
            preb_label = list()
            for j in range(preb.shape[1]):
                preb_label.append(np.argmax(preb[:, j], axis=0))
            no_repeat_blank_label = list()
            pre_c = preb_label[0]
            if pre_c != len(CHARS) - 1:
                no_repeat_blank_label.append(pre_c)
            for c in preb_label: # dropout repeat label and blank label
                if (pre_c == c) or (c == len(CHARS) - 1):
                    if c == len(CHARS) - 1:
                        pre_c = c
                    continue
                no_repeat_blank_label.append(c)
                pre_c = c
            preb_labels.append(no_repeat_blank_label)

        for i, label in enumerate(preb_labels):
            if args.show:
                show(imgs[i], label, targets[i])
            if len(label) != len(targets[i]):
                Tn_1 += 1
                continue
            if (np.asarray(targets[i]) == np.asarray(label)).all():
                Tp += 1
            else:
                Tn_2 += 1

    Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
    t2 = time.time()
    return Acc, (t2 - t1) / len(datasets)

In [43]:
args = get_parser_test()
tvmBaseline_accuracy, tvmBaseline_test_speed = Greedy_Decode_Eval_TVM(module, test_dataset, args)

# TVM Optimization

# TVM - Autotuning

In [44]:
from tvm import auto_scheduler

In [45]:
# Used to generate tuning_records.json

# tasks, task_weights = auto_scheduler.extract_tasks(mod, params, target)
# tuner = auto_scheduler.TaskScheduler(tasks, task_weights)
# tune_option = auto_scheduler.TuningOptions(
#     num_measure_trials=1000,
#     measure_callbacks=[auto_scheduler.RecordToFile("tuning_records.json")],
#     verbose=2,
# )
# tuner.tune(tune_option)

## Relay Quantization

In [46]:
with relay.quantize.qconfig(calibrate_mode="global_scale", global_scale=8.0):
    mod = relay.quantize.quantize(mod, params)

## Graph Level Transformation

In [47]:
from tvm.relay import transform

seq = tvm.transform.Sequential(
    [
        transform.InferType(),
        transform.FoldConstant(),
        transform.DeadCodeElimination(),
        transform.FoldScaleAxis(),
    ]
)
mod = seq(mod)

## Applying Results of Auto-tuning

In [48]:
with auto_scheduler.ApplyHistoryBest("tuning_records.json"):
    with tvm.transform.PassContext(opt_level=3, config={"relay.backend.use_auto_scheduler": True}):
        lib = relay.build(mod, target=target, params=params)

fused_nn_avg_pool2d_cast_cast_cast_multiply
Cannot find tuned schedules for target=llvm -keys=cpu -mtriple=x86_64-redhat-linux-gnu, workload_key=["13dee742f86444f84bbf7889ad0ee752", [100, 32, 20, 90, 4], [100, 32, 4, 18, 4]]. A fallback TOPI schedule is used, which may bring great performance regression or even compilation failure. Compute DAG info:
p0 = PLACEHOLDER [100, 32, 20, 90, 4]
pool_sum(ax0, ax1, ax2, ax3, ax4) += p0[ax0, ax1, ((ax2*5) + rv0), ((ax3*5) + rv1), ax4]
pool_avg(ax0, ax1, ax2, ax3, ax4) = (pool_sum[ax0, ax1, ax2, ax3, ax4]/(((min(((ax2*5) + 4), 19) - (ax2*5)) + 1)*((min(((ax3*5) + 4), 89) - (ax3*5)) + 1)))
T_cast(ax0, ax1, ax2, ax3, ax4) = uint8(pool_avg[ax0, ax1, ax2, ax3, ax4])
T_cast(ax0, ax1, ax2, ax3, ax4) = int32(T_cast[ax0, ax1, ax2, ax3, ax4])
T_cast(ax0, ax1, ax2, ax3, ax4) = float32(T_cast[ax0, ax1, ax2, ax3, ax4])
compile_engine_const() = 0.42173f
T_multiply(ax0, ax1, ax2, ax3, ax4) = (T_cast[ax0, ax1, ax2, ax3, ax4]*compile_engine_const[])

fused_nn_avg

In [49]:
from tvm.contrib import graph_executor

dtype = "float32"
module = graph_executor.GraphModule(lib["default"](dev))

In [50]:
args = get_parser_test()
tvmOptimized_accuracy, tvmOptimized_test_speed = Greedy_Decode_Eval_TVM(module, test_dataset, args)

# Testing inference speeds, model size and accuracy

In [51]:
def get_model_size(model):
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p")
    os.remove('temp.p')
    return size / 1e6  # Size in MB

original_size = get_model_size(lprnet)
pruned_size = get_model_size(pruned_lprnet)
quantized_size = get_model_size(quantized_lprnet)

print(f"Original model size: {original_size:.2f} MB")
print(f"Pruned model size: {pruned_size:.2f} MB")
print(f"Quantized model size: {quantized_size:.2f} MB")

Original model size: 1.81 MB
Pruned model size: 1.84 MB
Quantized model size: 0.50 MB


In [52]:
# Find the highest and lowest values
lowest_size = min(original_size, pruned_size, quantized_size)
highest_size = max(original_size, pruned_size, quantized_size)

# Calculate the speedup
size_percent_change = ((lowest_size - highest_size) / highest_size) * 100
print(f"The percentage decrease in model size is: {abs(size_percent_change):.2f}%")

The percentage decrease in model size is: 72.60%


In [53]:
# Find the highest and lowest values
lowest_time = min(baseline_test_speed, pruned_test_speed, quantized_test_speed, tvmBaseline_test_speed, tvmOptimized_test_speed)
highest_time = max(baseline_test_speed, pruned_test_speed, quantized_test_speed, tvmBaseline_test_speed, tvmOptimized_test_speed)

# Calculate the speedup
time_percent_change = ((lowest_time - highest_time) / highest_time) * 100

print(f"The percentage decrease in inference time is: {abs(time_percent_change):.2f}%")

The percentage decrease in inference time is: 91.42%


In [60]:
highest_accuracy = max(quantized_accuracy, tvmBaseline_accuracy, tvmOptimized_accuracy)

accuracy_percent_change = ((highest_accuracy - baseline_accuracy) / baseline_accuracy) * 100

print(f"The percentage decrease in accuracy is: {abs(accuracy_percent_change):.2f}%")

The percentage decrease in accuracy is: 16.65%
